In [1]:
import os
os.chdir("../")
from dataclasses import dataclass
from pathlib import Path
from zipfile import ZipFile
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories
from DeepClassifier.entities import DataIngestionConfig
import tensorflow as tf

In [2]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

## Configuration Manager

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ) -> None:
        """Inits ConfigurationManager.

        Args:
            config_file_path (Path, optional): Path of the config.yaml file.
                Defaults to the constant CONFIG_FILE_PATH.
            params_file_path (Path, optional): Path of the params.yaml file.
                Defaults to the constant PARAMS_FILE_PATH.
        """
        # Getting information in the config.yaml and params.yaml file
        self.config = read_yaml(yaml_file_path=config_file_path)
        self.params = read_yaml(yaml_file_path=params_file_path)
        
        # Creating the 'artifacts' directory
        create_directories(paths_of_directories=[self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """Creates and returns PrepareBaseModelConfig.

        Returns:
            PrepareBaseModelConfig: The PrepareBaseModelConfig.
        """
        # Getting the values in the `prepare_base_model` key of the config.yaml
        # file
        config = self.config.prepare_base_model
        
        # Creating the directory 'artifacts/prepare_base_model'
        create_directories(paths_of_directories=[config.root_dir])
        
        # Creating and returning `PrepareBaseModelConfig`
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )
        return prepare_base_model_config

## Components

In [4]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        """Inits PrepareBaseModel.

        Args:
            config (PrepareBaseModelConfig): The PrepareBaseModelConfig.
        """
        self.config = config
    
    def get_base_model(self):
        # Getting the VGG16 model as the base model
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
        )
        
        # Getting the path of the base model
        base_model_path = self.config.base_model_path
        
        # Saving the base model
        self.save_model(model=self.model, path=base_model_path)
    
    @staticmethod
    def _prepare_full_model(
        model,
        classes,
        freeze_all,
        freeze_till,
        learning_rate,
    ):
        if freeze_all:  # if `freeze_all` is `True`
            # we freeze the weights of all the layers of the model
            for layer in model.layers:
                layer.trainable = False
        
        # If `freeze_all` is `False` and `freeze_till` is not `None` and > 0
        elif (freeze_till is not None) and (freeze_till > 0):
            # we freeze the weights of all the layers except the last
            # `freeze_till` layers
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
        
        # Flattening the output of the above base model
        flatten_in = tf.keras.layers.Flatten()(model.output)
        
        # Creating the output layer for the full model
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax",
        )(flatten_in)
        
        # Creating and compiling the full model
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction,
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        # Getting the summary of the full model
        full_model.summary()
        
        # Returning the full model
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
        )
        
        self.save_model(
            model=self.full_model,
            path=self.config.updated_base_model_path,
        )
    
    @staticmethod
    def save_model(model: tf.keras.Model, path: Path):
        model.save(path)

In [5]:
try:
    config = ConfigurationManager()

    prepare_base_model_config = config.get_prepare_base_model_config()

    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

58889256/58889256 [==============================] - 11s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                             